## Build project

In [ ]:
# Xóa build cũ và tạo mới
!rm -rf build && mkdir build

# Build với CMake
!cmake -B build -S . && cmake --build build -- -j$(nproc)

## Download CIFAR-10 dataset

In [ ]:
import os
import urllib.request
import tarfile

data_dir = "data"
os.makedirs(data_dir, exist_ok=True)

cifar_folder = os.path.join(data_dir, "cifar-10-batches-bin")
if not os.path.exists(cifar_folder):
    print("Downloading CIFAR-10 dataset...")
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz"
    filename = os.path.join(data_dir, "cifar-10-binary.tar.gz")

    urllib.request.urlretrieve(url, filename)

    print("Extracting...")
    with tarfile.open(filename, "r:gz") as tar:
        tar.extractall(data_dir)

    os.remove(filename)
    print("Done!")
else:
    print("CIFAR-10 dataset already exists.")

# Check
!ls -la data/cifar-10-batches-bin/

## Tạo thư mục output

In [ ]:
!mkdir -p output

## Chạy chương trình với CPU mode

In [ ]:
!./build/extract_features ./data/cifar-10-batches-bin/ ./output --cpu

## Chạy chương trình với GPU mode (Baseline)

In [ ]:
!./build/extract_features ./data/cifar-10-batches-bin/ ./output --gpu

## Chạy chương trình với GPU mode (v1)

In [ ]:
!./build/extract_features ./data/cifar-10-batches-bin/ ./output --gpu_v1

## Chạy chương trình với GPU mode (v2)

In [ ]:
!./build/extract_features ./data/cifar-10-batches-bin/ ./output --gpu_v2

In [ ]:
import subprocess
import sys

print("Installing RAPIDS cuML...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "cuml", "-q"])

from cuml.svm import SVC as cuMLSVC
import time

print("Training SVM with RAPIDS cuML (GPU)...")
start_time = time.time()
clf_cuml = cuMLSVC(kernel='rbf', C=10, gamma='scale')
clf_cuml.fit(X_train_scaled, y_train)
train_time = time.time() - start_time
print(f"RAPIDS cuML Training time: {train_time:.2f} seconds")

# Evaluate
y_pred_cuml = clf_cuml.predict(X_test_scaled)
acc_cuml = accuracy_score(y_test, y_pred_cuml)
print(f"RAPIDS cuML Accuracy: {acc_cuml:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_cuml))

## SVM (RAPIDS cuML - GPU)

In [ ]:
import numpy as np
from pathlib import Path
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import time

# Load features from output folder
output_folder = Path("./output")
feature_size = 8 * 8 * 128  # 8192 latent features

X_train = np.fromfile(output_folder / "train_features.bin", dtype=np.float32).reshape(-1, feature_size)
y_train = np.fromfile(output_folder / "train_labels.bin", dtype=np.uint8)
X_test = np.fromfile(output_folder / "test_features.bin", dtype=np.float32).reshape(-1, feature_size)
y_test = np.fromfile(output_folder / "test_labels.bin", dtype=np.uint8)

# Normalize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Training SVM with LibSVM (CPU)...")
start_time = time.time()
clf_libsvm = SVC(kernel='rbf', C=10, gamma='scale')
clf_libsvm.fit(X_train_scaled, y_train)
train_time = time.time() - start_time
print(f"LibSVM Training time: {train_time:.2f} seconds")

# Evaluate
y_pred = clf_libsvm.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)
print(f"LibSVM Accuracy: {acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

## SVM (LibSVM - CPU)